In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator 

Using TensorFlow backend.


In [2]:
num_classes = 2
img_rows = 32
img_cols = 32
batch_size = 32

In [3]:
train_data_dir = "C:\\Users\\Girish\\Desktop\\Python\\CNN\\data\\train"
validation_data_dir = 'C:\\Users\\Girish\\Desktop\\Python\\CNN\\data\\validation'
test_data_dir = 'C:\\Users\\Girish\\Desktop\\Python\\CNN\\data\\test'

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  width_shift_range = 0.3,
                                  height_shift_range = 0.3,
                                  rotation_range = 0.3,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(directory = train_data_dir,
                                                   target_size = (img_rows,img_cols),
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

Found 2400 images belonging to 2 classes.


In [6]:
validation_generator = validation_datagen.flow_from_directory(directory = validation_data_dir,
                                                             target_size = (img_rows,img_cols),
                                                             batch_size = batch_size,
                                                             shuffle = False,
                                                             class_mode = 'categorical')

Found 800 images belonging to 2 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(directory = test_data_dir,
                                                 target_size = (img_rows,img_cols),
                                                 batch_size = batch_size,
                                                 shuffle = False,
                                                 class_mode = 'categorical')

Found 800 images belonging to 2 classes.


In [8]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,BatchNormalization

In [9]:
def convnet():
    model = Sequential([
    Conv2D(32,(3,3),padding = 'same',activation = 'relu',input_shape = (img_rows,img_cols,3)),
    Conv2D(32,(3,3),activation = 'relu'),
    MaxPooling2D(pool_size=(2,2),strides =2),
    BatchNormalization(),
    
    Conv2D(64,(3,3),padding = 'same',activation = 'relu'),
    Conv2D(64,(3,3),activation = 'relu'),
    MaxPooling2D(pool_size=(2,2),strides=2),
    BatchNormalization(),

    Conv2D(64,(3,3),padding = 'same',activation = 'relu'),
    Conv2D(64,(3,3),activation = 'relu'),
    MaxPooling2D(pool_size=(2,2),strides=2),
    BatchNormalization(),
    
    Conv2D(128,(3,3),padding = "same",activation="relu"),
    Conv2D(128,(3,3),padding="same",activation="relu"),
    MaxPooling2D(pool_size=(2,2),strides=2),
    BatchNormalization(),
    
    Flatten(),
    Dense(128,activation='relu'),
    BatchNormalization(),
    Dense(num_classes,activation='softmax')
    ])
    return model

In [10]:
model= convnet()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         

In [11]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [12]:

earlystop = EarlyStopping(monitor = 'val_accuracy',
                         min_delta = 0,
                         patience = 3,
                         verbose = 1,
                         restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                             factor = 0.2,
                             patience = 3,
                             verbose = 1,
                             min_delta = 0.0001)
callbacks = [earlystop,reduce_lr]

In [14]:
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [15]:
nb_train_samples = 2400
nb_validation_samples = 800
epochs = 100

In [16]:
history = model.fit_generator(train_generator,
                             epochs = epochs,
                             steps_per_epoch = nb_train_samples // batch_size,
                             validation_data = validation_generator,
                             validation_steps = nb_validation_samples // batch_size,
                             )

Epoch 1/100
75/75 [==============================] - 16s 209ms/step - loss: 0.8386 - accuracy: 0.5271 - val_loss: 0.7646 - val_accuracy: 0.5000
Epoch 2/100
75/75 [==============================] - 14s 192ms/step - loss: 0.7019 - accuracy: 0.5725 - val_loss: 0.5031 - val_accuracy: 0.5000
Epoch 3/100
75/75 [==============================] - 14s 190ms/step - loss: 0.6929 - accuracy: 0.5688 - val_loss: 0.3025 - val_accuracy: 0.5000
Epoch 4/100
75/75 [==============================] - 14s 190ms/step - loss: 0.6647 - accuracy: 0.6150 - val_loss: 0.4327 - val_accuracy: 0.5663
Epoch 5/100
75/75 [==============================] - 14s 192ms/step - loss: 0.6564 - accuracy: 0.6204 - val_loss: 0.6425 - val_accuracy: 0.6313
Epoch 6/100
75/75 [==============================] - 14s 193ms/step - loss: 0.6202 - accuracy: 0.6521 - val_loss: 0.1808 - val_accuracy: 0.5300
Epoch 7/100
75/75 [==============================] - 14s 191ms/step - loss: 0.6060 - accuracy: 0.6821 - val_loss: 0.4193 - val_accuracy:

In [17]:
scores = model.evaluate_generator(validation_generator,steps = nb_validation_samples // batch_size+1,verbose = 1)
print("\n Test Result: %.3f loss : %.3f" %(scores[1]*100 ,scores[0]))

26/26 [==============================] - 2s 94ms/step

 Test Result: 90.385 loss : 0.086


In [18]:
model.save('car_classifier_model.h5')

In [19]:
new_model = keras.models.load_model('car_classifier_model.h5')

In [20]:
y_pred = new_model.predict_generator(validation_generator,nb_validation_samples // batch_size)

In [21]:
y_pred_label = np.argmax(y_pred,axis = 1)

In [22]:
nb_test_samples = 800
test_pred = model.predict_generator(test_generator,nb_test_samples // batch_size)

In [23]:
test_pred_label = np.argmax(test_pred,axis=1)

In [24]:
from sklearn.metrics import confusion_matrix,classification_report
print("Confusion validation Matrix")
print(confusion_matrix(validation_generator.classes,y_pred_label))
print("Classification Validation Report")
class_labels = validation_generator.class_indices
class_labels = {v: k for k,v in class_labels.items()}
classes = list(class_labels.values())
print(classification_report(validation_generator.classes,y_pred_label,target_names = classes))

Confusion validation Matrix
[[381  19]
 [ 60 340]]
Classification Validation Report
              precision    recall  f1-score   support

       swift       0.86      0.95      0.91       400
      wagonr       0.95      0.85      0.90       400

    accuracy                           0.90       800
   macro avg       0.91      0.90      0.90       800
weighted avg       0.91      0.90      0.90       800



In [25]:
print("Confusion Test Matrix")
print(confusion_matrix(test_generator.classes,test_pred_label))
print('Classification Test Report')
class_labels = test_generator.class_indices
class_labels = {z : k for k,z in class_labels.items()}
classes = list(class_labels.values())
print(classification_report(test_generator.classes,test_pred_label,target_names = classes))

Confusion Test Matrix
[[378  22]
 [ 58 342]]
Classification Test Report
              precision    recall  f1-score   support

       swift       0.87      0.94      0.90       400
      wagonr       0.94      0.85      0.90       400

    accuracy                           0.90       800
   macro avg       0.90      0.90      0.90       800
weighted avg       0.90      0.90      0.90       800



In [26]:
from keras.preprocessing import image
import os
import cv2
from os import listdir
from os.path import isfile,join
import re

def draw_test(name,pred,im,true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im,160,0,0,500,cv2.BORDER_CONSTANT,value = BLACK)
    cv2.putText(expanded_image,"predicted - "+pred,(20,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.putText(expanded_image,"true -"+true_label,(20,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow(name,expanded_image)
    
def getRandomImage(path,img_width,img_height):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path+"/"+path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path,target_size = (img_width,img_height)),final_path,path_class

img_width,img_height = 32,32

files = []
predictions = []
true_labels = []

for i in range(0, 10):
    path = 'C:\\Users\\Girish\\Desktop\\Python\\CNN\\data\\test'
    img,final_path,true_label = getRandomImage(path,img_width,img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x*1./255
    x = np.expand_dims(x,axis = 0)
    images = np.vstack([x])
    classes = model.predict_classes(images,batch_size = 10)
    predictions.append(classes)
    
for i in range(0,len(files)):
    image = cv2.imread((files[i]))
    draw_test("Prediction",class_labels[predictions[i][0]],image,true_labels[i])
    cv2.waitKey(0)
    
cv2.destroyAllWindows()